In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [5]:
links = ['https://worldofbuzz.com/latest-news/',
        'https://worldofbuzz.com/social/',
        'https://worldofbuzz.com/people/',
        'https://worldofbuzz.com/lifestyle/',
        'https://worldofbuzz.com/culture/',
        'https://worldofbuzz.com/opinion/']

In [ ]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 


for i in tqdm(links):

    x = 1

    while True:

        while True:
            try:
                r = requests.get(f'{i}page/{x}',headers = headers)
                break
            except Exception as e:
                print(e)
                time.sleep(1.0)
                
        soup = BeautifulSoup(r.text, "lxml")
        
        div = soup.find_all('li', attrs = {"class":"mvp-blog-story-wrap"})

        if len(div) == 0:
            break

        for link in soup.find_all('li', attrs = {"class":"mvp-blog-story-wrap"}):
            
            href = link.find('a').get('href')
            hrefs.append(href)
                
        
        x = x + 1
    

In [7]:
len(hrefs)

15288

In [ ]:
hrefs = list(set(hrefs))

In [ ]:
with open('wob-link.json', 'a') as f:
    json.dump(hrefs, f)

In [6]:
with open('wob-link.json') as f:
    hrefs = json.load(f)

In [8]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="mvp-post-title left entry-title").text
        h = soup.find('div', attrs = {"class":"dable-content-wrapper"})
        p = [p.text for p in h.find_all("p") if len(p.text.split(" "))>5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [9]:
max_workers = 10

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('worldofbuzz.jsonl', 'a',encoding='utf8') as f:
                json.dump(result, f,ensure_ascii=False)
                f.write('\n')

 39%|█████████████████████████████████████████▉                                                                 | 5993/15288 [36:27<1:02:09,  2.49it/s]

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 44%|███████████████████████████████████████████████▍                                                           | 6782/15288 [41:19<1:05:08,  2.18it/s]

HTTPSConnectionPool(host='worldofbuzz.com', port=443): Max retries exceeded with url: /heineken-malaysia-allowed-to-operate-during-mco-but-pas-urges-govt-to-revoke-permission/ (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1131)')))


 55%|███████████████████████████████████████████████████████████▉                                                 | 8414/15288 [51:00<30:22,  3.77it/s]

...

 71%|███████████████████████████████████████████████████████████████████████████▌                              | 10891/15288 [1:05:32<28:15,  2.59it/s]

HTTPSConnectionPool(host='worldofbuzz.com', port=443): Max retries exceeded with url: /just-in-malaysia-records-new-high-with-5725-covid-19-cases-16-deaths/ (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1131)')))


 92%|█████████████████████████████████████████████████████████████████████████████████████████████████▎        | 14040/15288 [1:24:11<05:23,  3.86it/s]

error in link:https://worldofbuzz.com/malaysian-girls-stop-their-skincare-routine-for-7-days/
'NoneType' object has no attribute 'text'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 15288/15288 [1:31:35<00:00,  2.78it/s]
